https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.pdf

In [1]:
import pandas as pd

In [22]:
dfAll = pd.read_csv('./data/NCSBE/ncvoter_Wake.csv')
numRecs = dfAll.shape[0]

In [26]:
fromID = 0
toID = 0+10000
address_set = dfAll.iloc[fromID:toID,2:-1].dropna(how='any')
address_set.to_csv('./scratch/WakeSet{}.csv'.format(fromID),index=True,header=False)

In [5]:
df = pd.read_csv('./data/NCSBE/ncvoter_Wake.csv',nrows=10000)
df.to_csv('./scratch/WakeSet1.csv',index=False)

In [4]:
#census
theURL = 'https://geocoding.geo.census.gov/geocoder/locations/addressbatch --output geocoderesult.csv'

array(['27604', '28412', '27518', ..., '275021058', '90069', '286077742'], dtype=object)